In [26]:
import perceval as pcvl
import numpy as np
from perceval.algorithm import Analyzer, Sampler
from perceval import Circuit, BS, BasicState, Processor, NoiseModel, Detector

## Create simple 3 mode circuit

In [44]:
circuit = pcvl.Circuit(3)

pcvl.pdisplay(circuit, output_format=pcvl.Format.TEXT)

  
    
0:────:0 (depth 0)
    
    
1:────:1 (depth 0)
    
    
2:────:2 (depth 0)
    
  


## Create processor to simulate

In [ ]:
# Define circuit simulation with exact probabilites

proc = pcvl.Processor("SLOS", circuit)
proc.with_input(pcvl.BasicState([0, 5, 0]))

sampler = Sampler(proc)
prob_distribution = sampler.probs()
results = prob_distribution['results']
# overlap = results[pcvl.BasicState([0, 1, 0])]

print("Probabilities of the states:")
print(results)
# -----------------------------

# Define circuit simulation with noise
proc = pcvl.Processor("SLOS", circuit)
proc.with_input(pcvl.BasicState([0, 5, 0]))

proc.min_detected_photons_filter(pcvl.BasicState([0, 5, 0]).n)
proc.noise = NoiseModel(indistinguishability=0.9)
sampler = Sampler(proc)
prob_distribution = sampler.probs()
results = prob_distribution['results']

print("Probabilities of the states with noise:")
print(results)
# ------------------------------

# Define circuit simulation with noise and detectors
proc = pcvl.Processor("SLOS", circuit)
proc.with_input(pcvl.BasicState([0, 5, 0]))

proc.min_detected_photons_filter(pcvl.BasicState([0, 5, 0]).n)
proc.noise = NoiseModel(indistinguishability=0.9)

for m in range(proc.m):
    proc.add(m, Detector.threshold())

sampler = Sampler(proc)
prob_distribution = sampler.probs()
results = prob_distribution['results']

print("Probabilities of the states with noise and detectors:")
print(results)
# ------------------------------


Probabilities of the states:
{
  |0,5,0>: 1.0
}
Probabilities of the states with noise:
{
  |0,5,0>: 1.0
}
Probabilities of the states with noise and detectors:
{
  
}
[warning] Unable to normalize a distribution with only null probabilities
[warning] Unable to normalize a distribution with only null probabilities


In [35]:
# Define circuit simulation with sample size
proc = pcvl.Processor("SLOS", circuit)
proc.with_input(pcvl.BasicState([0, 5, 0]))

sampler = Sampler(proc)
sample_count = sampler.sample_count(1000)
results = sample_count['results']

print(results)

{
  |0,5,0>: 1000
}


In [28]:
# 1. Define the Circuit
# A simple 2-mode circuit with a Beam Splitter
circuit = pcvl.Circuit(2).add(0, BS())
print("Circuit defined:")
pcvl.pdisplay(circuit)

# 2. Define the Input Source
# Input state: one photon in each mode
input_state = BasicState([1, 1])
print(f"\nInput State: {input_state}")

# 3. Define a Noise Model
# Example: 10% photon loss probability on each mode
noise = NoiseModel(indistinguishability=0.1)
print(f"\nNoise Model: {noise}")

# 4. Define Detector Characteristics (Optional but good practice)
# Example: Detectors with 95% quantum efficiency
detector = Detector(quantum_efficiency=0.95)
print(f"\nDetector Model: {detector}")

# 5. Create the Processor
# Using the "SLOS" backend
processor = Processor(backend="SLOS",
                      circuit=circuit,
                      source_input=input_state, # Directly pass input state here
                      noise=noise,
                      detector=detector)

print(f"\nProcessor created with backend: {processor.backend_name}")
print(f"Processor input state set: {processor.source._input_state}")
print(f"Processor noise model set: {processor.noise}")
print(f"Processor detector model set: {processor.detector}")


# --- Example Usage ---
# You can now use this processor with algorithms like Sampler or Analyzer

# Example with Sampler:
print("\nSampling from the processor:")
sampler = Sampler(processor)
counts = sampler.sample_count(1000) # Simulate 1000 shots
print("Sampled counts:")
print(counts['results'])

Circuit defined:

Input State: |1,1>

Noise Model: {'indistinguishability': 0.1}


TypeError: __init__() got an unexpected keyword argument 'quantum_efficiency'

In [9]:
import perceval as pcvl
from perceval.algorithm import Analyzer, Sampler

# Fixed parameters
SIZE = 6
INPUT_FOCK = pcvl.BasicState([0, 0, 1, 0, 0, 0])
PHASES = [0] * (SIZE * (SIZE - 1) // 2)  # All phases set to 0
BACKEND = "SLOS"

# Build the circuit
circuit = pcvl.Circuit(SIZE)
param_idx = 0

#TODO: Add source
#TODO: Add noise

# Implement the rectangular mesh (Clements layout)
for layer in range(SIZE):
    start_idx = layer % 2  # Alternating between even and odd layers
    
    for i in range(start_idx, SIZE - 1, 2):
        # Add a Mach-Zehnder interferometer
        circuit.add((i, i+1), pcvl.BS())
        circuit.add((i+1,), pcvl.PS(phi=PHASES[param_idx]))
        circuit.add((i, i+1), pcvl.BS())
        param_idx += 1
        
        if param_idx >= len(PHASES):
            break

print("Circuit:", circuit)

# 2) Wrap it in a Processor on the Fock backend
proc = pcvl.Processor("SLOS", circuit)

# 3) Inject one photon into mode 1 (BasicState([0,1,0]))
proc.with_input(INPUT_FOCK)

# 4) Sample 1000 shots
sampler = Sampler(proc)

counts = sampler.sample_count(10000)

print(counts['results'])

probs = sampler.probs(10000000)

print(probs['results'])



Circuit: <perceval.components.linear_circuit.Circuit object at 0x13b2d3520>
{
  |0,0,0,1,0,0>: 10000
}
{
  |0,0,0,1,0,0>: 1.0
}


In [24]:
from perceval.algorithm import Sampler

# Get amplitude
backend = pcvl.BackendFactory().get_backend(backend_name="Naive")
backend.set_circuit(circuit)
backend.set_input_state(pcvl.BasicState(INPUT_FOCK))

# quantum_processor.with_input(pcvl.LogicalState([0,0,0]))

sampler = Sampler(backend)

output_distribution = sampler.probs()["results"]
pcvl.pdisplay(output_distribution, precision=1e-2, max_v = 4)

AttributeError: 'NaiveBackend' object has no attribute 'is_remote'

In [ ]:
import perceval as pcvl
from perceval.components import catalog
# from perceval.components.noise import SourceLossModel
from perceval.components.unitary_components import Clements

# Define single-photon source parameters from thesis
brightness = 47e3  # photons/s/mW (used for reference, not in API)
efficiency = 0.30  # heralding efficiency η_c
indistinguishability = 0.96  # HOM visibility ~ indistinguishability

# Create a noise model
# noise_model = SourceLossModel(efficiency=efficiency)

# Define Perceval single-photon source with noise and indistinguishability
# sps = pcvl.Source(noise_model=noise_model,
#                   indistinguishability=indistinguishability)

# Create 2 indistinguishable photons in spatial modes 0 and 1
# (matching 2-photon interference requirement in thesis)
input_state = pcvl.BasicState("|1,1,0,0,0,0>")  # 6-mode spatial encoding

# Define a 6x6 Clements interferometer (unitary photonic circuit)
# Here we create a random unitary for demonstration
import numpy as np
from perceval.utils import random_unitary

U = random_unitary(6)
circuit = Clements(U)

# Create the processor
proc = pcvl.Processor("SLOS", 6)
proc.add(circuit)

# Set the input state
proc.with_input(input_state)

# Set the sources (replace with actual source model if needed)
# proc.set_sources([sps for _ in range(6)])  # 6 sources, only 2 used here

# Simulate the experiment
simulator = pcvl.AlgoProcessor(proc)
results = simulator.execute()

# Display results
for output_state, probability in results.items():
    print(f"{output_state} -> {probability:.4f}")

ImportError: cannot import name 'Clements' from 'perceval.components.unitary_components' (/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/perceval/components/unitary_components.py)

In [ ]:
import perceval as pcvl
from perceval.components import catalog
# from perceval.components.noise import SourceLossModel
from perceval.components.unitary_components import Clements

# Define single-photon source parameters from thesis
brightness = 47e3  # photons/s/mW (used for reference, not in API)
efficiency = 0.30  # heralding efficiency η_c
indistinguishability = 0.96  # HOM visibility ~ indistinguishability

# Create a noise model
# noise_model = SourceLossModel(efficiency=efficiency)

# Define Perceval single-photon source with noise and indistinguishability
# sps = pcvl.Source(noise_model=noise_model,
#                   indistinguishability=indistinguishability)

# Create 2 indistinguishable photons in spatial modes 0 and 1
# (matching 2-photon interference requirement in thesis)
input_state = pcvl.BasicState("|1,1,0,0,0,0>")  # 6-mode spatial encoding

# Define a 6x6 Clements interferometer (unitary photonic circuit)
# Here we create a random unitary for demonstration
import numpy as np
from perceval.utils import random_unitary

U = random_unitary(6)
circuit = Clements(U)

# Create the processor
proc = pcvl.Processor("SLOS", 6)
proc.add(circuit)

# Set the input state
proc.with_input(input_state)

# Set the sources (replace with actual source model if needed)
# proc.set_sources([sps for _ in range(6)])  # 6 sources, only 2 used here

# Simulate the experiment
simulator = pcvl.AlgoProcessor(proc)
results = simulator.execute()

# Display results
for output_state, probability in results.items():
    print(f"{output_state} -> {probability:.4f}")

ImportError: cannot import name 'Clements' from 'perceval.components.unitary_components' (/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/perceval/components/unitary_components.py)

In [ ]:
import perceval as pcvl
from perceval.components import catalog
# from perceval.components.noise import SourceLossModel
from perceval.components.unitary_components import Clements

# Define single-photon source parameters from thesis
brightness = 47e3  # photons/s/mW (used for reference, not in API)
efficiency = 0.30  # heralding efficiency η_c
indistinguishability = 0.96  # HOM visibility ~ indistinguishability

# Create a noise model
# noise_model = SourceLossModel(efficiency=efficiency)

# Define Perceval single-photon source with noise and indistinguishability
# sps = pcvl.Source(noise_model=noise_model,
#                   indistinguishability=indistinguishability)

# Create 2 indistinguishable photons in spatial modes 0 and 1
# (matching 2-photon interference requirement in thesis)
input_state = pcvl.BasicState("|1,1,0,0,0,0>")  # 6-mode spatial encoding

# Define a 6x6 Clements interferometer (unitary photonic circuit)
# Here we create a random unitary for demonstration
import numpy as np
from perceval.utils import random_unitary

U = random_unitary(6)
circuit = Clements(U)

# Create the processor
proc = pcvl.Processor("SLOS", 6)
proc.add(circuit)

# Set the input state
proc.with_input(input_state)

# Set the sources (replace with actual source model if needed)
# proc.set_sources([sps for _ in range(6)])  # 6 sources, only 2 used here

# Simulate the experiment
simulator = pcvl.AlgoProcessor(proc)
results = simulator.execute()

# Display results
for output_state, probability in results.items():
    print(f"{output_state} -> {probability:.4f}")

ImportError: cannot import name 'Clements' from 'perceval.components.unitary_components' (/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/perceval/components/unitary_components.py)

In [4]:
import perceval as pcvl
from perceval.components import catalog
# from perceval.components.noise import SourceLossModel
from perceval.components.unitary_components import Clements

# Define single-photon source parameters from thesis
brightness = 47e3  # photons/s/mW (used for reference, not in API)
efficiency = 0.30  # heralding efficiency η_c
indistinguishability = 0.96  # HOM visibility ~ indistinguishability

# Create a noise model
# noise_model = SourceLossModel(efficiency=efficiency)

# Define Perceval single-photon source with noise and indistinguishability
# sps = pcvl.Source(noise_model=noise_model,
#                   indistinguishability=indistinguishability)

# Create 2 indistinguishable photons in spatial modes 0 and 1
# (matching 2-photon interference requirement in thesis)
input_state = pcvl.BasicState("|1,1,0,0,0,0>")  # 6-mode spatial encoding

# Define a 6x6 Clements interferometer (unitary photonic circuit)
# Here we create a random unitary for demonstration
import numpy as np
from perceval.utils import random_unitary

U = random_unitary(6)
circuit = Clements(U)

# Create the processor
proc = pcvl.Processor("SLOS", 6)
proc.add(circuit)

# Set the input state
proc.with_input(input_state)

# Set the sources (replace with actual source model if needed)
# proc.set_sources([sps for _ in range(6)])  # 6 sources, only 2 used here

# Simulate the experiment
simulator = pcvl.AlgoProcessor(proc)
results = simulator.execute()

# Display results
for output_state, probability in results.items():
    print(f"{output_state} -> {probability:.4f}")

ImportError: cannot import name 'Clements' from 'perceval.components.unitary_components' (/Users/anani/Code/uq-qnn/.venv/lib/python3.9/site-packages/perceval/components/unitary_components.py)

In [ ]:






# # Get some example amplitudes and probabilities
# example_output = [0, 1, 0, 1, 0, 1]

# # Get amplitude
# backend = pcvl.BackendFactory().get_backend(BACKEND)
# backend.set_circuit(circuit)
# backend.set_input_state(pcvl.BasicState(INPUT_FOCK))
# amplitude = backend.prob_amplitude(pcvl.BasicState(example_output))
# print(f"\nAmplitude for output {example_output}: {amplitude}")

# # Get probability
# probability = backend.probability(pcvl.BasicState(example_output))
# print(f"Probability for output {example_output}: {probability}")

# # Get full distribution
# processor = pcvl.Processor(BACKEND, circuit)
# analyzer = Analyzer(processor, [pcvl.BasicState(INPUT_FOCK)], '*')

# # 1. COMPUTE THE ANALYSIS
# analyzer.compute()  # <-- This was missing!

# # 2. ACCESS THE RESULTS
# print("\nFull probability distribution:")
# distribution = analyzer.distribution  # Get ProbabilityDistribution object
# print(distribution)
# # Print non-zero probabilities only (filter out 0% outcomes)
# for output_state, probability in distribution.items():
#     if probability > 1e-10:  # Adjust threshold as needed
#         print(f"{output_state}: {probability:.4f}")